In [328]:
# import statements
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import sklearn

In [329]:
# read in data
train_df = pd.read_csv("data/train.csv") 
test_df = pd.read_csv("data/test.csv")

In [330]:
# preview the datatest_df.head()
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [331]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [332]:
# see info about data
train_df.info()
print("----------------------------")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
----------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare

In [333]:
# a naive solution, ignoring data cleaning and other useful relationships



In [334]:
# set nan for age to the mean of the age column
train_df['Age'] = train_df.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))
test_df['Age'] = test_df.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))

# set nan fare to the mean of the fare column
train_df['Fare'] = train_df.groupby(['Pclass'])['Fare'].transform(lambda x: x.fillna(x.mean()))
test_df['Fare'] = test_df.groupby(['Pclass'])['Fare'].transform(lambda x: x.fillna(x.mean()))

# make nan for sex male since men are more likely to not
# survive, and most dont survive
train_df['Sex'] = train_df.groupby(['Pclass'])['Sex'].transform(lambda x: x.fillna('male'))
test_df['Sex'] = test_df.groupby(['Pclass'])['Sex'].transform(lambda x: x.fillna('male'))

#train_df['Age'] = train_df['Age'].astype(int)
#test_df['Age'] = test_df['Age'].astype(int)

#train_df.loc[train_df['Age'] <= 16, 'Age'] = 0
#train_df.loc[(train_df['Age'] > 16) & (train_df['Age'] <= 32), 'Age'] = 1
#train_df.loc[(train_df['Age'] > 32) & (train_df['Age'] <= 48), 'Age'] = 2
#train_df.loc[(train_df['Age'] > 48) & (train_df['Age'] <= 64), 'Age'] = 3
#train_df.loc[(train_df['Age'] > 64), 'Age'] = 4

#test_df.loc[ test_df['Age'] <= 16, 'Age'] = 0
#test_df.loc[(test_df['Age'] > 16) & (test_df['Age'] <= 32), 'Age'] = 1
#test_df.loc[(test_df['Age'] > 32) & (test_df['Age'] <= 48), 'Age'] = 2
#test_df.loc[(test_df['Age'] > 48) & (test_df['Age'] <= 64), 'Age'] = 3
#test_df.loc[(test_df['Age'] > 64), 'Age'] = 4



In [335]:
# make everything ints
train_df['Age'] = train_df['Age'].astype(int)
test_df['Age'] = test_df['Age'].astype(int)
train_df['Fare'] = train_df['Fare'].astype(int)
# test_df['Fare'] = test_df['Fare'].astype(int)
# set men to 1 and female to 0
train_df['Sex'] = train_df['Sex'].map(lambda s : 1 if s == 'male' else 0)
test_df['Sex'] = test_df['Sex'].map(lambda s : 1 if s == 'male' else 0)

# clean data
test_df1 = test_df.drop(['Ticket','Cabin', 'Embarked', 'Name'], axis=1)
train_df1 = train_df.dropna()
train_df1 = train_df.drop(['Ticket','Cabin', 'Embarked', 'Name', 'PassengerId'], axis=1)
train_df1.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22,1,0,7
1,1,1,0,38,1,0,71
2,1,3,0,26,0,0,7
3,1,1,0,35,1,0,53
4,0,3,1,35,0,0,8


In [336]:

# define data
X_train = train_df1.drop("Survived",axis=1)
Y_train = train_df1["Survived"]
X_test  = test_df1.drop("PassengerId",axis=1).copy()

In [337]:
# test random forest classifier
random_forest = RandomForestClassifier(n_estimators=100)
#random_forest = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, max_features='sqrt', min_samples_split=5)

random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

0.95735129068462399

In [338]:
# gradient boost
#gradient_boost = GradientBoostingClassifier(n_estimators=100,loss='exponential',max_features='log2')
gradient_boost = GradientBoostingClassifier(n_estimators=100)
gradient_boost.fit(X_train, Y_train)

Y_pred_2 = gradient_boost.predict(X_test)

gradient_boost.score(X_train, Y_train)

0.89450056116722787

In [339]:
# extra tree
#extra_tree = ExtraTreesClassifier(n_estimators=100,criterion='gini',max_depth=10,max_features='log2',min_samples_split=10)
extra_tree = ExtraTreesClassifier(n_estimators=100)
extra_tree.fit(X_train, Y_train)

Y_pred_3 = extra_tree.predict(X_test)

extra_tree.score(X_train, Y_train)

0.95735129068462399

In [340]:
# kNearest Neighbors
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 7,weights='distance')
knn.fit(X_train, Y_train)
Y_pred_4 = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

95.74

In [341]:
# Ada boost
ada_boost = AdaBoostClassifier(n_estimators=100,algorithm='SAMME')

ada_boost.fit(X_train, Y_train)

Y_pred_5 = ada_boost.predict(X_test)

ada_boost.score(X_train, Y_train)

0.82379349046015715

In [342]:
# Gaussian Naive Bayes

gaussian = GaussianNB()

gaussian.fit(X_train, Y_train)

Y_pred_6 = gaussian.predict(X_test)

gaussian.score(X_train, Y_train)

0.79236812570145898

In [343]:
# set which alg you want to predict
Y_pred = Y_pred_6

In [344]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('titanic.csv', index=False)

In [345]:
# attempt at improving our solution
df = train_df.append(test_df)

df['Age'] = df.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))

# set nan fare to the mean of the fare column
df['Fare'] = df.groupby(['Pclass'])['Fare'].transform(lambda x: x.fillna(x.mean()))

# set nan fare to the mean of the fare column
df['Survived'] = df.groupby(['Pclass'])['Survived'].transform(lambda x: x.fillna(0))

# make nan for sex male since men are more likely to not
# survive, and most dont survive
df['Sex'] = df.groupby(['Pclass'])['Sex'].transform(lambda x: x.fillna(1))

# make everything ints
df['Age'] = df['Age'].astype(int)

df['Fare'] = df['Fare'].astype(int)

df.head()



,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22,NaN,S,7,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171
1,38,C85,C,71,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599
2,26,NaN,S,7,"Heikkinen, Miss. Laina",0,3,3,0,0,1.0,STON/O2. 3101282
3,35,C123,S,53,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,0,1,1.0,113803
4,35,NaN,S,8,"Allen, Mr. William Henry",0,5,3,1,0,0.0,373450


In [346]:
# Family
# Set a family column to a bool.
# True if the passenger has family on the boat, F if not
df['FamilySize'] = df['SibSp'] + df['Parch']
df

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,FamilySize
0,22,NaN,S,7,"Braund, Mr. Owen Harris",0,1,3,1,1,0.0,A/5 21171,1
1,38,C85,C,71,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,1,1.0,PC 17599,1
2,26,NaN,S,7,"Heikkinen, Miss. Laina",0,3,3,0,0,1.0,STON/O2. 3101282,0
3,35,C123,S,53,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,0,1,1.0,113803,1
4,35,NaN,S,8,"Allen, Mr. William Henry",0,5,3,1,0,0.0,373450,0
5,25,NaN,Q,8,"Moran, Mr. James",0,6,3,1,0,0.0,330877,0
6,54,E46,S,51,"McCarthy, Mr. Timothy J",0,7,1,1,0,0.0,17463,0
7,2,NaN,S,21,"Palsson, Master. Gosta Leonard",1,8,3,1,3,0.0,349909,4
8,27,NaN,S,11,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,0,0,1.0,347742,2
9,14,NaN,C,30,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,0,1,1.0,237736,1


In [347]:
# train_df = train_df.drop(['SibSp','Parch'], axis=1)
# test_df = test_df.drop(['SibSp','Parch'], axis=1)
df['Title'] = df['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
# map embark chars to ints
def embark(e):
    if e == "S":
        return 0
    elif e == "C":
        return 1
    elif e == "Q":
        return 2
    else:
        return 3
          
df['Embarked'] = df['Embarked'].map(lambda e: embark(e))
# create columns for if the passenger is in a family or is alone
df['Alone'] = df['FamilySize'].map(lambda s : 1 if s == 0 else 0)
df['Family'] = df['FamilySize'].map(lambda s : 1 if s>=3 else 0)

df['Cabin'] = df['Cabin'].map(lambda c : 0 if pd.isnull(c) else ord(str(c[0])))

titles_feature = pd.get_dummies(df['Title'],prefix='Title')
df = pd.concat([df,titles_feature],axis=1)

df = df.drop(['Name','Ticket', 'Title'], axis=1)

# df.info()


In [348]:
train = df[0:890]
test = df[891:1309]

In [355]:
X_train = train.drop(["Survived", "PassengerId"],axis=1)
Y_train = train["Survived"]
X_test  = test.drop(["Survived", "PassengerId"],axis=1).copy()

# gradient boost
#gradient_boost = GradientBoostingClassifier(n_estimators=100,loss='exponential',max_features='log2')
gradient_boost = GradientBoostingClassifier(n_estimators=100)
gradient_boost.fit(X_train, Y_train)

Y_pred_2 = gradient_boost.predict(X_test)

gradient_boost.score(X_train, Y_train)
X_test


,Age,Cabin,Embarked,Fare,Parch,Pclass,Sex,SibSp,FamilySize,Alone,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,34,0,2,7,0,3,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,47,0,0,7,0,3,0,1,1,0,...,0,0,0,0,0,1,0,0,0,0
2,62,0,2,9,0,2,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,27,0,0,8,0,3,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,22,0,0,12,1,3,0,1,2,0,...,0,0,0,0,0,1,0,0,0,0
5,14,0,0,9,0,3,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
6,30,0,2,7,0,3,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
7,26,0,0,29,1,2,1,1,2,0,...,0,0,0,0,1,0,0,0,0,0
8,18,0,1,7,0,3,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
9,21,0,0,24,0,3,1,2,2,0,...,0,0,0,0,1,0,0,0,0,0


In [382]:
train

,Age,Cabin,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess
0,22,0,0,7,0,1,3,1,1,0.0,...,0,0,0,0,1,0,0,0,0,0
1,38,67,1,71,0,2,1,0,1,1.0,...,0,0,0,0,0,1,0,0,0,0
2,26,0,0,7,0,3,3,0,0,1.0,...,0,1,0,0,0,0,0,0,0,0
3,35,67,0,53,0,4,1,0,1,1.0,...,0,0,0,0,0,1,0,0,0,0
4,35,0,0,8,0,5,3,1,0,0.0,...,0,0,0,0,1,0,0,0,0,0
5,25,0,2,8,0,6,3,1,0,0.0,...,0,0,0,0,1,0,0,0,0,0
6,54,69,0,51,0,7,1,1,0,0.0,...,0,0,0,0,1,0,0,0,0,0
7,2,0,0,21,1,8,3,1,3,0.0,...,1,0,0,0,0,0,0,0,0,0
8,27,0,0,11,2,9,3,0,0,1.0,...,0,0,0,0,0,1,0,0,0,0
9,14,0,1,30,0,10,2,0,1,1.0,...,0,0,0,0,0,1,0,0,0,0


In [386]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=200)
clf = clf.fit(X_train, Y_train)

features = pd.DataFrame()
features['feature'] = X_train.columns
features['importance'] = clf.feature_importances_
features

model = SelectFromModel(clf, prefit=True)
training = model.transform(X_train)
training.shape


(890, 8)

In [387]:

X_test = X_test.fillna(method='ffill')

testing = model.transform(X_test)
testing.shape


(418, 8)

In [388]:

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
forest = RandomForestClassifier(max_features='sqrt')

parameter_grid = {'max_depth':[5],'n_estimators': [220],'criterion': ['gini','entropy']}

grid_search = GridSearchCV(forest,
                           param_grid=parameter_grid,
                           cv=StratifiedKFold(Y_train, n_folds=5))

grid_search.fit(training, Y_train)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

pipeline = grid_search
output = pipeline.predict(testing).astype(int)
df_output = pd.DataFrame()
df_output['PassengerId'] = test['PassengerId']
df_output['Survived'] = output
df_output[['PassengerId','Survived']].to_csv('output1.csv',index=False)



Best score: 0.812359550562
Best parameters: {'n_estimators': 220, 'criterion': 'gini', 'max_depth': 5}


In [389]:
training

array([[22,  0,  7, ...,  0,  1,  0],
       [38, 67, 71, ...,  0,  0,  1],
       [26,  0,  7, ...,  1,  0,  0],
       ..., 
       [19, 66, 30, ...,  1,  0,  0],
       [25,  0, 23, ...,  1,  0,  0],
       [26, 67, 30, ...,  0,  1,  0]])

In [353]:
submission = pd.DataFrame({
    "Survived": Y_pred_2,
    "PassengerId": test["PassengerId"]
    })
submission.to_csv('titanic.csv', index=False)